In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pasm1e_9
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pasm1e_9
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=9ce11203c3fbc00e3d1ab475b32a26c369bae2bb9a92f34661210b18fb9c3ce3
  Stored in directory: /tmp/pip-ephem-wheel-cache-m1rhf7vr/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [53]:
%%cuda --name MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__device__ bool result = true;

__global__ void findIS (Coloring* col, GraphStruct *str, uint * weights, bool *currentIS, bool *X) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if(X[idx])
			return;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx]){
			X[idx] = true;
			return;
	}

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!X[neighID] && !col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		currentIS[idx] = true;
    X[idx] = true;
    for (uint j = 0; j < deg; j++) {
        uint neighID = str->neighs[offset + j];
        X[neighID] = true;
    }
	}
}

__global__ void checkX(GraphStruct *str, bool* X) {
		uint idx = threadIdx.x + blockDim.x * blockIdx.x;
		if (idx >= str->nodeSize)
				return;
		result = false;
    if(X[idx] == false)
      result = true;
}

__global__ void colorer(Coloring* col, GraphStruct *str, bool *currentIS){
  uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

  if(currentIS[idx])
      col->coloring[idx] = col->numOfColors;
	else
	 		col->uncoloredNodes = true;
}

void maximalIS(Coloring * col, GraphStruct *str, uint * weigths, bool * currentIS, bool * X){
		dim3 threads (THREADxBLOCK);
		dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
		bool result_h = true;

		while(result_h){
        findIS <<< blocks, threads >>> (col, str, weigths, currentIS, X);
				checkX <<< blocks, threads >>> (str, X);
				cudaDeviceSynchronize();
				cudaMemcpyFromSymbol(&result_h, result, 1*sizeof(bool));
    }
}

Coloring* graphColoring(GraphStruct *str){
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMallocManaged((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);
	cudaDeviceSynchronize();



	bool* currentMIS;
	cudaMalloc((void**) &currentMIS, n * sizeof(bool));
	cudaMemset(currentMIS, false, n);

	bool* X;
	cudaMalloc((void**) &X, n * sizeof(bool));
	cudaMemset(X, false, n);

	col->numOfColors = 0;

	print_d <<< 1, 1 >>> (str, true);

	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		cudaMemset(currentMIS, false, n);
		cudaMemset(X, false, n);

		maximalIS(col, str, weigths, currentMIS, X);
    colorer <<< blocks, threads >>> (col, str, currentMIS);
		cudaDeviceSynchronize();
	}
	 printf("Pesi: ");
	 for(int i = 0; i < str->nodeSize; i++){
			printf("%d ", weigths[i]);
	}
	 printf("\n");

  cudaFree(states);
	cudaFree(weigths);
  return col;
}

__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx >= n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}


'File written in /content/src/MIS.cu'

In [8]:
%%cuda --name test_MIS.cu

#include "/content/drive/MyDrive/graph/coloring.h"

using namespace std;

int main(void) {
	unsigned int n = 6;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	printf("Colorazione: ");
	for(int i = 0; i < n; i++){
			printf("%d ", col->coloring[i]);
	}

	return EXIT_SUCCESS;
}

'File written in /content/src/test_MIS.cu'

In [54]:
!nvcc -dc -D__CDPRT_SUPPRESS_SYNC_DEPRECATION_WARNING src/test_MIS.cu /content/src/MIS.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_MIS.o MIS.o graph.o graph_d.o -o test_MIS
!./test_MIS

** Graph (num node: 6, num edges: 6)
  node(0)[3]-> 1 2 3 
  node(1)[3]-> 0 3 5 
  node(2)[2]-> 0 3 
  node(3)[3]-> 0 1 2 
  node(4)[0]-> 
  node(5)[1]-> 1 

Pesi: 0 6 18 6 24 6 
3.377152 ms
Colorazione: 4 3 1 2 1 1 